## Bergfex Webscraping
<b> Milestone 2</b> 

Scraping the snow level data from meteocentrale.ch website in order to create Pandas dataframes that contain the snow level for 91 weather stations around Switzerland.
This code uses BeautifulSoup to parse the html tags. A for-loop iterates over each html tag and adds the corresponding information into empty lists. Afterwards the data is cleaned.
Finally, we add the GPS coordinates of each weather station, from two databases containing geodata of Swiss cities and ski resorts.

### Installations

In [1]:
# unhash and run the below line once
#conda install -c anaconda beautifulsoup4

### Imports

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# 1. Data Scraping: snow information

We use the same packages as in notebook I to scrape the meteocentrale website, which contains information for around 80 locations in Switzerland with local snow thickness measurements updated every day at 7 AM.

In [28]:
# Initializing the future colums of our dataframe with empty lists

snowlevel = []  # height in cm
location = []  # town and elevation of town

# Only scrape one page (no looping over several pages necessary as not so many data points available)
link = 'http://www.meteocentrale.ch/de/wetter/hitlisten/schneehoehen.html'
page = requests.get(link, timeout=10)
print(page.status_code)
soup = BeautifulSoup(page.content, "html.parser")  # bs4.BeautifulSoup object
hitlist = soup.findAll('table', {'class': 'hitlist'})  #bs4.element.ResultSet

# If '200' then the scraping was successful

200


# 2. Extracting the necessary information
Village Name, Elevation, Snow Level

In [29]:
# Get location name
location_item = hitlist[0].findAll('a')
location.append([info.get_text().strip() for info in location_item])
loc = location[0]

# Get snowlevel
snowlevel_item = hitlist[0].findAll('td', {'class': 'value'}) 
snowlevel.append([info.get_text().strip() for info in snowlevel_item])
snow = snowlevel[0]

# Combine into DF
heights_df = pd.DataFrame({'location': loc,'snowlevel': snow})
heights_df

,location,snowlevel
0,"Grimsel-Hospiz, 1980 m",264 cm
1,"Weissfluhjoch, 2690 m",236 cm
2,"Corvatsch, 3315 m",164 cm
3,"Gütsch/Andermatt, 2282 m",144 cm
4,"Corviglia, 2497 m",78 cm
...,...,...
78,"Zernez, 1478 m",0 cm
79,"Zollikofen, 553 m",0 cm
80,"Zürich-Affoltern, 443 m",0 cm
81,"Zürich-Flughafen, 432 m",0 cm


# 3. Data Cleaning

## Clean numerical data

In [30]:
# Remove cm, convert to 'int'
heights_df['snowlevel_in_cm']=pd.Series(heights_df['snowlevel']).str.replace(" cm", '')
heights_df['snowlevel_in_cm']=pd.Series(heights_df['snowlevel_in_cm']).astype(int)

# Split location into 'village' and 'elevation of village' and merge with previous DF
split_loc = pd.Series(heights_df['location']).str.split(',',n=2,expand = True)
merged_df = pd.merge(heights_df, split_loc, left_index=True, right_index=True)

# Drop unused columns, rename final columns, remove unit, sort columns
intermediate_df = merged_df.iloc[:,[2,3,4]].copy()
intermediate_df.columns = ['snowlevel_in_cm', 'location', 'height_in_m']
intermediate_df['height_in_m']=pd.Series(intermediate_df['height_in_m']).str.replace(" m", '')
snow_level_df = pd.DataFrame(intermediate_df, columns = ['location', 'height_in_m', 'snowlevel_in_cm'])
snow_level_df.head()

,location,height_in_m,snowlevel_in_cm
0,Grimsel-Hospiz,1980,264
1,Weissfluhjoch,2690,236
2,Corvatsch,3315,164
3,Gütsch/Andermatt,2282,144
4,Corviglia,2497,78


## Write CSV

In [31]:
# change the file_path to your path if necessary
file_path = '../data/'
snow_level_df.to_csv(file_path + 'snow_level_test.csv', index = False)

# 4. Adding Coordinates

## Matching the snow level observation station with GPS coordinates

We need to get the coordinates for each snow observation station (for which we have the name of the locality)

We use two sources (all coordinates are given in geographical coordinate system EPSG:4326/WGS84): 

 - 1/ from World Cities Database, which contains most large cities in Switzerland and geographic info saved as swiss_cities.csv
       https://simplemaps.com/data/ch-cities
 - 2/ Ski resort coordinates, for the smaller but more relevant localities in mountaineous areas. Those were collected as an initial csv file from skiresort.info (this dataset is not available online anymore), and missing data was collected manually from Wikipedia.org. The resulting file was saved as ski.csv

In [38]:
# cleaning of dataframes: this step needs to be adapted to data sources and csv content. 
# Here the swiss_cities files were overwritten with the cleaned up version, so no need to run the following steps

file_path = '../data/' # change to your local folder

# swiss_cities should have index, city, lat, lng columns (if not, uncomment the cleaning lines)
swiss_cities = pd.read_csv((file_path + 'swiss_cities.csv')) 
# ski_resorts should have index, city, lat, lng columns (if not, uncomment the cleaning lines)
ski_resorts = pd.read_csv((file_path + 'ski.csv'))

## swiss_cities cleaning (uncomment if necessary)
# swiss_cities = swiss_cities.drop(columns ={'iso2', 'country', 'capital', 'population', 'population_proper', 'admin_name'})
swiss_cities = swiss_cities[['city', 'lat', 'lng']]

## ski_resorts cleaning
# ski_resorts should have index, lng, lat, city columns
ski_resorts = ski_resorts.rename(columns = {'X':'lng', 'Y':'lat', 'Name': 'city'})
ski_resorts = ski_resorts.drop(columns = {'description'})
ski_resorts = ski_resorts[['city', 'lat', 'lng']]


# merging the two dataframes by concatenating them vertically
cities_ski = pd.concat([swiss_cities, ski_resorts], ignore_index=True)


In [39]:
cities_ski.info() # should have 177 rows 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   city    177 non-null    object 
 1   lat     177 non-null    float64
 2   lng     177 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.3+ KB


In [40]:
# saving the dataframe as city_coord
cities_ski.to_csv(file_path +'cities_ski_coord.csv', index = False) # has city, lat long in it

Now we still need to match the snow observation locations to the place's coordinated. We do that by merging the corresponding dataframes on the location names. Data will be lost when the names are not matching exaclty. As the number of errors is quite small (and we didn't know about regex yet), we dealt with that problem by filling the missing data by hand, and the complete dataframe for notebook is saved as snow_coordinates.csv

In [43]:
# merging city_coordinates with snow data to create snow_coordinates_demo
# this is a dummy df that only contains limited data for demonstration purposes. 
# In the next notebook we will use the actual snow_coordinates df

snow_level_coord = cities_ski.merge(snow_level_df, left_on = 'city', right_on= 'location')
snow_level_coord = snow_level_coord.drop_duplicates('city', keep='first').drop('city', 1)
snow_level_coord = snow_level_coord[['location', 'lat', 'lng', 'height_in_m', 'snowlevel_in_cm']]
snow_level_coord

,location,lat,lng,height_in_m,snowlevel_in_cm
0,La Chaux-de-Fonds,47.099600,6.829600,1019,1
1,Chur,46.852100,9.529700,555,0
2,Wädenswil,47.230300,8.672200,463,0
3,Nyon,46.382000,6.238900,430,0
4,Davos,46.809100,9.839800,1590,19
5,Sursee,47.174200,8.108100,510,0
6,Altdorf,46.880600,8.639400,449,0
7,Arosa,46.789611,9.638778,1878,48
9,Braunwald,46.938250,8.987139,None,11
11,Engelberg,46.790917,8.387722,1035,0


## Write CSV

In [37]:
# change the file_path to your path if necessary
#file_path = '../data/'
snow_level_coord.to_csv(file_path + 'snow_coordinates_demo.csv', index = False)

# CONTINUE IN NOTEBOOK III